# はじめる前に

19

## 型?

In [ ]:
:type (!!)

In [ ]:
:type (+)

20

## kind?

In [ ]:
:kind Int

In [ ]:
:kind (->)

In [ ]:
:kind Int -> Int

In [ ]:
:kind Maybe

In [ ]:
:kind Maybe Int

In [ ]:
:kind Num

21

## kindが違うとエラー

In [ ]:
:kind Maybe Either

22
# part1. 型から値へ

23

## Phantom type

In [ ]:
data Foo a = Foo { getFoo :: Int }

24

## Boolean型の情報を保持してみる

In [ ]:
data On
data Off

In [ ]:
class KnownFlag a where
    flagVal :: Foo a -> Bool

instance KnownFlag On where
    flagVal _ = True

instance KnownFlag Off where
    flagVal _ = False

In [ ]:
testFoo :: KnownFlag a => Foo a -> String
testFoo foo =
    (if flagVal foo then "On:" else "Off:") ++
    show (getFoo foo)

25

## つかってみよう

In [ ]:
testFoo (Foo 12 :: Foo On)

In [ ]:
testFoo (Foo 12 :: Foo Off)

26

## これでok?

In [ ]:
class KnownFlag a where
    flagVal ::  a -> Bool -- Foo aをaに

instance KnownFlag On where
    flagVal _ = True

instance KnownFlag Off where
    flagVal _ = False

In [ ]:
testFoo :: KnownFlag a => Foo a -> String
testFoo foo =
    (if flagVal (undefined :: a) then "On:" else "Off:") ++
    show (getFoo foo)

27

## ScopedTypeVariables

In [ ]:
:set -XScopedTypeVariables

In [ ]:
testFoo :: forall a. KnownFlag a => Foo a -> String
testFoo foo =
    (if flagVal (undefined :: a) then "On:" else "Off:") ++
    show (getFoo foo)

28

## これでok?

In [ ]:
fooOk :: Foo Bool
fooOk = Foo 3

testFoo fooOk

29

## DataKinds

In [ ]:
:set -XDataKinds

In [ ]:
:kind 'True

In [ ]:
:kind 'Nothing

In [ ]:
:kind 'Just 'True

30

## PolyKinds

In [ ]:
:module Data.Proxy
:kind Proxy

In [ ]:
Proxy :: Proxy Int

In [ ]:
Proxy :: Proxy 'True

In [ ]:
Proxy :: Proxy Maybe

31

## 完成

In [ ]:
:set -XKindSignatures

In [ ]:
data Foo (a :: Bool) = Foo { getFoo :: Int }

In [ ]:
class KnownBool (a :: Bool) where
    boolVal :: proxy a -> Bool

instance KnownBool 'True where
    boolVal _ = True

instance KnownBool 'False where
    boolVal _ = False

In [ ]:
testFoo :: forall a. KnownBool a => Foo a -> String
testFoo foo =
    (if boolVal (Proxy :: Proxy a) then "On:" else "Off:") ++
    show (getFoo foo)

In [ ]:
testFoo' :: KnownBool a => Foo a -> String
testFoo' foo = (if boolVal foo then "On:" else "Off:") ++ show (getFoo foo)

32

## 型レベルリテラル

In [ ]:
:module GHC.TypeLits Data.Proxy

In [ ]:
:kind 3

In [ ]:
:kind "foo"

In [ ]:
typeLitTest :: (KnownNat n, KnownSymbol s) => proxyN n -> proxyS s -> [String]
typeLitTest nat sym = replicate
    (fromIntegral $ natVal nat)
    (symbolVal sym)

In [ ]:
typeLitTest (Proxy :: Proxy 5) (Proxy :: Proxy "foo")

33

# part2. 型族

34

## 型族?

In [ ]:
:type 2 + (3 :: Int)

In [ ]:
:kind 2 + 3 -- iHaskellでは:kind!使えないっぽい?

35

## 長さ付きリスト

36

## スマートコンストラクタ?

In [ ]:
data Expr a
    = I Int
    | B Bool
    | Add (Expr a) (Expr a)
    | Eq  (Expr a) (Expr a)

In [ ]:
i :: Int -> Expr Int
i = I

In [ ]:
eval :: Expr a -> a
eval (I i) = i

37

## GADTs

In [ ]:
:set -XGADTs

In [ ]:
data Expr :: * -> * where
    I   :: Int  -> Expr Int
    B   :: Bool -> Expr Bool
    Add :: Expr Int -> Expr Int -> Expr Int
    Eq  :: Expr Int -> Expr Int -> Expr Bool

In [ ]:
eval :: Expr a -> a
eval (I i) = i

38

## 型レベル自然数リテラルで

In [ ]:
data Vector :: Nat -> * -> * where
    Nil  :: Vector 0 a
    (:-) :: a -> Vector n a -> Vector (n + 1) a

infixr 5 :- -- iHaskellでは効いてないっぽい……

In [ ]:
:t Nil

In [ ]:
:t (1 :: Int) :- Nil

In [ ]:
:t 2 :- ((1 :: Int) :- Nil)

39

## いけてない

In [ ]:
vtail :: Vector (n + 1) a -> Vector n a
vtail (_ :- b) = b

40

## ならば定義を変える!

In [ ]:
data Vector :: Nat -> * -> * where
    Nil  :: Vector 0 a
    (:-) :: a -> Vector (n - 1) a -> Vector n a

In [ ]:
:type Nil

In [ ]:
:type (1::Int) :- Nil

In [ ]:
:type 2 :- ((1::Int) :- Nil)

41

## やっぱりダメ

In [ ]:
vtail :: Vector (n + 1) a -> Vector n a
vtail (_ :- b) = b

42

## ペアノ数

In [ ]:
data Nat = Z | S Nat

type N0 = 'Z
type N1 = 'S 'Z
type N2 = 'S N1
type N3 = 'S N2

43

## ペアノ数で

In [ ]:
:set -XStandaloneDeriving

In [ ]:
data Vector :: Nat -> * -> * where
    Nil  :: Vector 'Z a
    (:-) :: a -> Vector n a -> Vector ('S n) a

deriving instance Show a => Show (Vector n a)

infixr 5 :-

vtail :: Vector ('S n) a -> Vector n a
vtail (_ :- b) = b

In [ ]:
:type vtail Nil

In [ ]:
vtail (1 :- (2 :- Nil))

45

## 型族を定義する

In [ ]:
:set -XTypeFamilies

In [ ]:
type family (a :: Nat) + (b :: Nat) :: Nat

type instance 'Z   + b = b
type instance 'S a + b = 'S (a + b)

In [ ]:
import Data.Word

type family   Fraction a
type instance Fraction Float  = Word32
type instance Fraction Double = Word64
type instance Fraction a      = Integer

46

## closedな型族

In [ ]:
type family a + b where
  'Z   + b = b
  'S a + b = 'S (a + b)

In [ ]:
import Data.Word

type family Fraction a where
    Fraction Float  = Word32
    Fraction Double = Word64
    Fraction a      = Integer

47

## (++)

In [ ]:
(+:+) :: Vector x a -> Vector y a -> Vector (x + y) a
Nil       +:+ ys = ys
(x :- xs) +:+ ys = x :- (xs +:+ ys)

In [ ]:
(5 :- (3 :- Nil)) +:+ (1 :- (2 :- Nil))

48

## reverse

In [ ]:
reverse :: Vector n a -> Vector n a
reverse l = rev l Nil
  where
    rev :: Vector k a -> Vector l a -> Vector (k + l) a
    rev Nil       a = a
    rev (x :- xs) a = rev xs (x :- a)

50

# part3. 等しさ

In [ ]:
:module Data.Type.Equality

In [ ]:
:info Refl

In [ ]:
:type sym

In [ ]:
:type trans

In [ ]:
:type castWith

In [ ]:
:type apply

In [ ]:
:type inner

In [ ]:
:type outer

53

## :~:上での演算

In [ ]:
addL :: proxy k -> n :~: m -> k + n :~: k + m
addL _ Refl = Refl

54

## 左単位元

In [ ]:
plusLZ :: proxy n -> 'Z + n :~: n
plusLZ _ = Refl

55

## 右単位元?

In [ ]:
plusRZ :: proxy n -> n + 'Z :~: n
plusRZ _ = Refl

57

## シングルトン

In [ ]:
data SNat :: Nat -> * where
    SZ :: SNat 'Z
    SS :: SNat n -> SNat ('S n)

deriving instance Show (SNat n)

In [ ]:
(.+) :: SNat x -> SNat y -> SNat (x + y)
SZ   .+ y = y
SS x .+ y = SS (x .+ y)

58

## 右単位元

In [ ]:
plusRZ :: SNat n -> n + 'Z :~: n
plusRZ  SZ    = Refl
plusRZ (SS n) = apply Refl (plusRZ n)

In [ ]:
plusRZ SZ

In [ ]:
plusRZ (SS $ SS SZ)

59

## 結合則

In [ ]:
plusAssoc :: SNat x -> SNat y -> SNat z -> (x + y) + z :~: x + (y + z)
plusAssoc  SZ    _ _ = Refl
plusAssoc (SS x) y z = apply Refl (plusAssoc x y z)

In [ ]:
sAndPlusOne :: SNat n -> 'S n :~: n + N1
sAndPlusOne  SZ    = Refl
sAndPlusOne (SS n) = apply Refl (sAndPlusOne n)

60

## n + S m :~: S (n + m)

In [ ]:
data Hole = Hole -- type hole使えない……つらい……

In [ ]:
plusSR :: SNat n -> SNat m -> n + 'S m :~: 'S (n + m)
plusSR n m =
    Refl
    `trans` Hole

In [ ]:
plusSR :: SNat n -> SNat m -> n + 'S m :~: 'S (n + m)
plusSR n m =
    Refl
    `trans` addL n (sAndPlusOne m)
    `trans` Hole

In [ ]:
plusSR :: SNat n -> SNat m -> n + 'S m :~: 'S (n + m)
plusSR n m =
    Refl
    `trans` addL n (sAndPlusOne m)
    `trans` sym (plusAssoc n m (SS SZ))
    `trans` Hole

In [ ]:
plusSR :: SNat n -> SNat m -> n + 'S m :~: 'S (n + m)
plusSR n m =
    Refl
    `trans` addL n (sAndPlusOne m)
    `trans` sym (plusAssoc n m (SS SZ))
    `trans` sym (sAndPlusOne (n .+ m))
    `trans` Hole

In [ ]:
plusSR :: SNat n -> SNat m -> n + 'S m :~: 'S (n + m)
plusSR n m =
    addL n (sAndPlusOne m)
    `trans` sym (plusAssoc n m (SS SZ))
    `trans` sym (sAndPlusOne (n .+ m))

In [ ]:
plusSR (SS SZ) (SS $ SS SZ)

61

## reverseの証明(1)

In [ ]:
sLength :: Vector n a -> SNat n
sLength  Nil      = SZ
sLength (_ :- as) = SS (sLength as)

In [ ]:
sLength (1 :- (2 :- Nil))

In [ ]:
asVectorSize :: n :~: m -> Vector n a :~: Vector m a 
asVectorSize Refl = Refl

In [ ]:
vReverse :: Vector n a -> Vector n a
vReverse l = castWith (asVectorSize $ plusRZ (sLength l)) $ rev l Nil
  where
    rev :: Vector k a -> Vector l a -> Vector (k + l) a
    rev Nil       a = a
    rev (x :- xs) a =
        castWith (asVectorSize $ plusSR (sLength xs) (sLength a)) $
        rev xs (x :- a)

In [ ]:
vReverse $ (1 :: Int) :- (2 :- Nil)